In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

from shapely.geometry import Point
from shapely import contains_xy
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from tqdm.notebook import trange

In [2]:
def get_mask(lon, lat, shape_gdf):
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    points = np.vstack((lon_grid.ravel(), lat_grid.ravel())).T
    mask = contains_xy(shape_gdf.union_all(), points[:, 0], points[:, 1])
    mask = np.float32(mask)
    mask[mask == 0] = np.nan
    return mask.reshape(lon_grid.shape)

In [3]:
all_selections = pd.read_csv("../data/big_river_selection.txt", sep="\t")
basin_list = all_selections["grdc_no"].tolist()
basins_shape = gpd.read_file("../data/shape/stationbasins.shp")

lon_series = np.load("../../2025_04_Params_Transplant/Data/forcing/lon.npy")
lat_series = np.load("../../2025_04_Params_Transplant/Data/forcing/lat.npy")

time_series_historical = pd.date_range(start="1850-01-01", end="2014-12-31", freq="MS")
time_series_future = pd.date_range(start="2015-01-01", end="2099-12-31", freq="MS")

In [18]:
element = "pr"
results_historical = np.full((len(time_series_historical), len(basin_list)), np.nan)
results_126        = np.full((len(time_series_future), len(basin_list)), np.nan)
results_245        = np.full((len(time_series_future), len(basin_list)), np.nan)
results_370        = np.full((len(time_series_future), len(basin_list)), np.nan)
results_585        = np.full((len(time_series_future), len(basin_list)), np.nan)

gcm_historical = np.load(f"../data/mme/{element}_bc/{element}_historical_bc_mme.npy").astype(np.float32)
gcm_126        = np.load(f"../data/mme/{element}_bc/{element}_ssp126_bc_mme.npy").astype(np.float32)
gcm_245        = np.load(f"../data/mme/{element}_bc/{element}_ssp245_bc_mme.npy").astype(np.float32)
gcm_370        = np.load(f"../data/mme/{element}_bc/{element}_ssp370_bc_mme.npy").astype(np.float32)
gcm_585        = np.load(f"../data/mme/{element}_bc/{element}_ssp585_bc_mme.npy").astype(np.float32)
for b in trange(len(basin_list), desc=f"Processing {element}"):
    basin = basin_list[b]
    # 找 shapefile 子集
    basin_shp = basins_shape[basins_shape["grdc_no"] == basin]
    polygon = basin_shp.geometry
    minx, miny, maxx, maxy = polygon.total_bounds
    # 流域上下左右边界
    basin_left_bound = minx - 0.2
    basin_right_bound = maxx + 0.5
    basun_top_bound = maxy + 0.5
    basin_bottom_bound = miny - 0.2
    # 寻找落在流域边界内的格点
    lon_loc = np.where((lon_series >= basin_left_bound) & (lon_series <= basin_right_bound))[0]
    lat_loc = np.where((lat_series >= basin_bottom_bound) & (lat_series <= basun_top_bound))[0]
    temp_lon = lon_series[lon_loc]
    temp_lat = lat_series[lat_loc]
    # 计算 mask
    basin_mask = get_mask(temp_lon, temp_lat, basin_shp)
    # 区域平均
    temp_gcm_historical = gcm_historical[:, lat_loc[0]:lat_loc[-1]+1, lon_loc[0]:lon_loc[-1]+1]
    temp_gcm_126        = gcm_126[:, lat_loc[0]:lat_loc[-1]+1, lon_loc[0]:lon_loc[-1]+1]
    temp_gcm_245        = gcm_245[:, lat_loc[0]:lat_loc[-1]+1, lon_loc[0]:lon_loc[-1]+1]
    temp_gcm_370        = gcm_370[:, lat_loc[0]:lat_loc[-1]+1, lon_loc[0]:lon_loc[-1]+1]
    temp_gcm_585        = gcm_585[:, lat_loc[0]:lat_loc[-1]+1, lon_loc[0]:lon_loc[-1]+1]
    basin_series_historical = np.nanmean(temp_gcm_historical * basin_mask, axis=(1, 2))
    basin_series_126        = np.nanmean(temp_gcm_126        * basin_mask, axis=(1, 2))
    basin_series_245        = np.nanmean(temp_gcm_245        * basin_mask, axis=(1, 2))
    basin_series_370        = np.nanmean(temp_gcm_370        * basin_mask, axis=(1, 2))
    basin_series_585        = np.nanmean(temp_gcm_585        * basin_mask, axis=(1, 2))
    results_historical[:, b] = basin_series_historical
    results_126[:, b]        = basin_series_126
    results_245[:, b]        = basin_series_245
    results_370[:, b]        = basin_series_370
    results_585[:, b]        = basin_series_585
results_historical_df = pd.DataFrame(results_historical, index=time_series_historical, columns=basin_list).resample("YS").sum(min_count=1)
results_historical_df.index.name = "Time"
results_historical_df.to_csv(f"../results/big_river_series/{element}_historical_series.txt", sep="\t", float_format="%.1f")

results_126_df = pd.DataFrame(results_126, index=time_series_future, columns=basin_list).resample("YS").sum(min_count=1)
results_126_df.index.name = "Time"
results_126_df.to_csv(f"../results/big_river_series/{element}_ssp126_series.txt", sep="\t", float_format="%.1f")

results_245_df = pd.DataFrame(results_245, index=time_series_future, columns=basin_list).resample("YS").sum(min_count=1)
results_245_df.index.name = "Time"
results_245_df.to_csv(f"../results/big_river_series/{element}_ssp245_series.txt", sep="\t", float_format="%.1f")

results_370_df = pd.DataFrame(results_370, index=time_series_future, columns=basin_list).resample("YS").sum(min_count=1)
results_370_df.index.name = "Time"
results_370_df.to_csv(f"../results/big_river_series/{element}_ssp370_series.txt", sep="\t", float_format="%.1f")

results_585_df = pd.DataFrame(results_585, index=time_series_future, columns=basin_list).resample("YS").sum(min_count=1)
results_585_df.index.name = "Time"
results_585_df.to_csv(f"../results/big_river_series/{element}_ssp585_series.txt", sep="\t", float_format="%.1f")

Processing pr:   0%|          | 0/1194 [00:00<?, ?it/s]

In [16]:
element = "soil_moisture"
results_historical = np.full((len(time_series_historical), len(basin_list)), np.nan)
results_126        = np.full((len(time_series_future), len(basin_list)), np.nan)
results_245        = np.full((len(time_series_future), len(basin_list)), np.nan)
results_370        = np.full((len(time_series_future), len(basin_list)), np.nan)
results_585        = np.full((len(time_series_future), len(basin_list)), np.nan)

gcm_historical = np.load(f"../data/mme/{element}/{element}_historical_mme.npy").astype(np.float32)
gcm_126        = np.load(f"../data/mme/{element}/{element}_ssp126_mme.npy").astype(np.float32)
gcm_245        = np.load(f"../data/mme/{element}/{element}_ssp245_mme.npy").astype(np.float32)
gcm_370        = np.load(f"../data/mme/{element}/{element}_ssp370_mme.npy").astype(np.float32)
gcm_585        = np.load(f"../data/mme/{element}/{element}_ssp585_mme.npy").astype(np.float32)
gcm_historical[0] = gcm_historical[1]
gcm_126[0]        = gcm_126[1]
gcm_245[0]        = gcm_245[1]
gcm_370[0]        = gcm_370[1]
gcm_585[0]        = gcm_585[1]
for b in trange(len(basin_list), desc=f"Processing {element}"):
    basin = basin_list[b]
    # 找 shapefile 子集
    basin_shp = basins_shape[basins_shape["grdc_no"] == basin]
    polygon = basin_shp.geometry
    minx, miny, maxx, maxy = polygon.total_bounds
    # 流域上下左右边界
    basin_left_bound = minx - 0.2
    basin_right_bound = maxx + 0.5
    basun_top_bound = maxy + 0.5
    basin_bottom_bound = miny - 0.2
    # 寻找落在流域边界内的格点
    lon_loc = np.where((lon_series >= basin_left_bound) & (lon_series <= basin_right_bound))[0]
    lat_loc = np.where((lat_series >= basin_bottom_bound) & (lat_series <= basun_top_bound))[0]
    temp_lon = lon_series[lon_loc]
    temp_lat = lat_series[lat_loc]
    # 计算 mask
    basin_mask = get_mask(temp_lon, temp_lat, basin_shp)
    # 区域平均
    temp_gcm_historical = gcm_historical[:, lat_loc[0]:lat_loc[-1]+1, lon_loc[0]:lon_loc[-1]+1]
    temp_gcm_126        = gcm_126[:, lat_loc[0]:lat_loc[-1]+1, lon_loc[0]:lon_loc[-1]+1]
    temp_gcm_245        = gcm_245[:, lat_loc[0]:lat_loc[-1]+1, lon_loc[0]:lon_loc[-1]+1]
    temp_gcm_370        = gcm_370[:, lat_loc[0]:lat_loc[-1]+1, lon_loc[0]:lon_loc[-1]+1]
    temp_gcm_585        = gcm_585[:, lat_loc[0]:lat_loc[-1]+1, lon_loc[0]:lon_loc[-1]+1]
    basin_series_historical = np.nanmean(temp_gcm_historical * basin_mask, axis=(1, 2))
    basin_series_126        = np.nanmean(temp_gcm_126        * basin_mask, axis=(1, 2))
    basin_series_245        = np.nanmean(temp_gcm_245        * basin_mask, axis=(1, 2))
    basin_series_370        = np.nanmean(temp_gcm_370        * basin_mask, axis=(1, 2))
    basin_series_585        = np.nanmean(temp_gcm_585        * basin_mask, axis=(1, 2))
    results_historical[:, b] = basin_series_historical
    results_126[:, b]        = basin_series_126
    results_245[:, b]        = basin_series_245
    results_370[:, b]        = basin_series_370
    results_585[:, b]        = basin_series_585
results_historical_df = pd.DataFrame(results_historical, index=time_series_historical, columns=basin_list).resample("YS").sum(min_count=1)
results_historical_df.index.name = "Time"
results_historical_df.to_csv(f"../results/big_river_series/{element}_historical_series.txt", sep="\t", float_format="%.1f")

results_126_df = pd.DataFrame(results_126, index=time_series_future, columns=basin_list).resample("YS").sum(min_count=1)
results_126_df.index.name = "Time"
results_126_df.to_csv(f"../results/big_river_series/{element}_ssp126_series.txt", sep="\t", float_format="%.1f")

results_245_df = pd.DataFrame(results_245, index=time_series_future, columns=basin_list).resample("YS").sum(min_count=1)
results_245_df.index.name = "Time"
results_245_df.to_csv(f"../results/big_river_series/{element}_ssp245_series.txt", sep="\t", float_format="%.1f")

results_370_df = pd.DataFrame(results_370, index=time_series_future, columns=basin_list).resample("YS").sum(min_count=1)
results_370_df.index.name = "Time"
results_370_df.to_csv(f"../results/big_river_series/{element}_ssp370_series.txt", sep="\t", float_format="%.1f")

results_585_df = pd.DataFrame(results_585, index=time_series_future, columns=basin_list).resample("YS").sum(min_count=1)
results_585_df.index.name = "Time"
results_585_df.to_csv(f"../results/big_river_series/{element}_ssp585_series.txt", sep="\t", float_format="%.1f")

Processing soil_moisture:   0%|          | 0/1194 [00:00<?, ?it/s]